In [1]:
import os
import dxcam_cpp as dxcam
from src.utils.windowtools import (
    fuzzy_window_search,
    calculate_aspect_ratio,
    check_aspect_ratio_validity,
    get_monitor_number_from_coords,
    normalise_coords_to_monitor
)
from src.utils.helpers import (
    pre_process,
    pre_process_distbox,
)
from src.models import get_model, get_default_model_type, get_model_info
import matplotlib.pyplot as plt
from easyocr import Reader
import numpy as np
import cv2
import tkinter as tk
import threading
import time as systime
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

🔧 Model Configuration: Default model type set to 'optimized'


In [2]:
coords = fuzzy_window_search("asphalt")

monitor_id = get_monitor_number_from_coords(coords)

normalised_coords = normalise_coords_to_monitor(coords, monitor_id)

aspect_ratio = calculate_aspect_ratio(normalised_coords)

check_aspect_ratio_validity(aspect_ratio)
print(coords)

[(0, 0, 2560, 1392)]
1
The aspect ratio is reasonable.
(0, 0, 2560, 1392)


In [3]:
# Global vars
camera = dxcam.create(device_idx=0, output_idx=monitor_id)
capturing = True
time = 0
elapsed_ms = 0
percentage = 0
race_in_progress = False  # New variable to track race state

# Inference time tracking
inference_times = []
total_loops = 0
avg_inference_time = 0

reader = Reader(['en'], gpu=True)  # Still needed for DIST detection

In [4]:
# Grab a frame from the camera
window = camera.grab()

# Extract coordinates from the coords variable
x1, y1, x2, y2 = normalised_coords

capture_coords = (x1, y1, x2, int(y1 + (y2 - y1) / 3.4))

camera.start(region=capture_coords, target_fps=90)

In [5]:
def start_capturing():
    global capturing
    capturing = True

def stop_capturing():
    global capturing
    capturing = False

def toggle_pin():
    global root, is_pinned
    is_pinned = not is_pinned
    if is_pinned:
        root.wm_attributes("-topmost", True)
        pin_button.config(text="📌 Unpin", bg="#ff6b6b")
    else:
        root.wm_attributes("-topmost", False)
        pin_button.config(text="📌 Pin", bg="#4ecdc4")

def update_ui():
    global time, elapsed_ms, percentage, avg_inference_time, inference_times, race_in_progress
    
    # Update labels efficiently (only if values changed)
    time_label.config(text=f"Time: {time}")
    elapsed_label.config(text=f"Loop: {elapsed_ms:.1f}ms")
    
    # Smart status display based on race_in_progress
    if race_in_progress and percentage and percentage != "0%":
        # Extract numeric value for progress bar
        try:
            progress_value = int(percentage.replace('%', ''))
            progress_bar.config(value=progress_value)
            progress_label.config(text=f"{progress_value}%")
        except:
            progress_value = 0
            progress_bar.config(value=0)
            progress_label.config(text="0%")
        
        percentage_label.config(text=f"Distance: {percentage}", fg="#2ecc71")
        status_label.config(text="Race In Progress", fg="#2ecc71")
    else:
        progress_bar.config(value=0)
        progress_label.config(text="--")
        percentage_label.config(text="Distance: --", fg="#95a5a6")
        if race_in_progress:
            status_label.config(text="Searching For Race...", fg="#f39c12")
        else:
            status_label.config(text="Race Not Started", fg="#95a5a6")
    
    # Performance metrics
    if inference_times:
        current_inference = inference_times[-1] if inference_times else 0
        inference_label.config(text=f"Inference: {current_inference:.1f}ms")
        avg_inference_label.config(text=f"Average: {avg_inference_time:.1f}ms")
    else:
        inference_label.config(text="Inference: --")
        avg_inference_label.config(text="Average: --")

    # Schedule next update at 11ms (90 FPS) for ultra-responsive UI
    root.after(11, update_ui)

def create_ui():
    global root, time_label, elapsed_label, percentage_label, status_label
    global avg_inference_label, inference_label, pin_button, is_pinned
    global progress_bar, progress_label
    
    root = tk.Tk()
    root.title("ALU Timing Tool")
    root.geometry("500x240")
    root.resizable(False, False)
    
    # Set up the window style
    root.configure(bg="#2c3e50")
    is_pinned = False
    
    # Header with pin button
    header_frame = tk.Frame(root, bg="#2c3e50")
    header_frame.pack(fill="x", padx=10, pady=5)
    
    title_label = tk.Label(header_frame, text="ALU Timing Tool", 
                          font=("Helvetica", 12, "bold"), fg="#ecf0f1", bg="#2c3e50")
    title_label.pack(side="left")
    
    pin_button = tk.Button(header_frame, text="📌 Pin", command=toggle_pin, 
                          bg="#4ecdc4", fg="white", font=("Helvetica", 8),
                          relief="flat", padx=8, pady=2)
    pin_button.pack(side="right")
    
    # Main content area
    content_frame = tk.Frame(root, bg="#2c3e50")
    content_frame.pack(fill="both", expand=True, padx=10, pady=5)
    
    # Top section - horizontal layout
    top_frame = tk.Frame(content_frame, bg="#2c3e50")
    top_frame.pack(fill="x", pady=(0, 10))
    
    # Left side - Controls and Status
    left_frame = tk.Frame(top_frame, bg="#34495e", width=200)
    left_frame.pack(side="left", fill="y", padx=(0, 10))
    left_frame.pack_propagate(False)
    
    # Controls section
    controls_title = tk.Label(left_frame, text="Controls", 
                             font=("Helvetica", 10, "bold"), fg="#bdc3c7", bg="#34495e")
    controls_title.pack(pady=(10, 5))
    
    # Control buttons (stacked vertically)
    start_button = tk.Button(left_frame, text="▶️ Start", command=start_capturing, 
                            bg="#27ae60", fg="white", font=("Helvetica", 10, "bold"),
                            relief="flat", padx=10, pady=8, width=15)
    start_button.pack(pady=2)

    stop_button = tk.Button(left_frame, text="⏹️ Stop", command=stop_capturing, 
                           bg="#e74c3c", fg="white", font=("Helvetica", 10, "bold"),
                           relief="flat", padx=10, pady=8, width=15)
    stop_button.pack(pady=2)
    
    # Status section below controls
    status_frame = tk.Frame(left_frame, bg="#2c3e50", relief="flat", bd=1)
    status_frame.pack(fill="x", pady=(15, 10), padx=5)
    
    status_label = tk.Label(status_frame, text="Race Not Started", 
                           font=("Helvetica", 10, "bold"), fg="#95a5a6", bg="#2c3e50")
    status_label.pack(pady=8)
    
    # Right side - Metrics
    right_frame = tk.Frame(top_frame, bg="#34495e")
    right_frame.pack(side="right", fill="both", expand=True)
    
    # Metrics title
    metrics_title = tk.Label(right_frame, text="Performance Metrics", 
                            font=("Helvetica", 10, "bold"), fg="#bdc3c7", bg="#34495e")
    metrics_title.pack(pady=(10, 10))
    
    # Loop timing
    elapsed_label = tk.Label(right_frame, text=f"Loop: {elapsed_ms:.1f}ms", 
                            font=("Helvetica", 11), fg="#ecf0f1", bg="#34495e")
    elapsed_label.pack(pady=3)
    
    # Inference timing
    inference_label = tk.Label(right_frame, text="Inference: --", 
                              font=("Helvetica", 11), fg="#ecf0f1", bg="#34495e")
    inference_label.pack(pady=3)
    
    avg_inference_label = tk.Label(right_frame, text="Average: --", 
                                  font=("Helvetica", 11), fg="#ecf0f1", bg="#34495e")
    avg_inference_label.pack(pady=3)
    
    # Time (placeholder for future use)
    time_label = tk.Label(right_frame, text=f"Time: {time}", 
                         font=("Helvetica", 11), fg="#ecf0f1", bg="#34495e")
    time_label.pack(pady=3)
    
    # Distance percentage
    percentage_label = tk.Label(right_frame, text="Distance: --", 
                               font=("Helvetica", 11, "bold"), fg="#95a5a6", bg="#34495e")
    percentage_label.pack(pady=3)
    
    # Progress bar section at bottom
    progress_frame = tk.Frame(content_frame, bg="#2c3e50")
    progress_frame.pack(fill="x", pady=(0, 5))
    
    # Progress bar container with indicators
    progress_container = tk.Frame(progress_frame, bg="#2c3e50")
    progress_container.pack(fill="x", pady=5)
    
    # Start indicator (0%)
    start_label = tk.Label(progress_container, text="0%", 
                          font=("Helvetica", 10, "bold"), fg="#ecf0f1", bg="#2c3e50")
    start_label.pack(side="left", padx=(0, 10))
    
    # Import ttk for progress bar
    try:
        from tkinter import ttk
        style = ttk.Style()
        style.theme_use('clam')
        style.configure("Custom.Horizontal.TProgressbar", 
                       background='#2ecc71',
                       troughcolor='#34495e',
                       borderwidth=0,
                       lightcolor='#2ecc71',
                       darkcolor='#2ecc71')
        
        progress_bar = ttk.Progressbar(progress_container, 
                                      style="Custom.Horizontal.TProgressbar",
                                      length=300, mode='determinate',
                                      maximum=99)
        progress_bar.pack(side="left", fill="x", expand=True, padx=(0, 10))
        
    except ImportError:
        # Fallback if ttk is not available
        progress_bar = tk.Frame(progress_container, bg="#34495e", height=20)
        progress_bar.pack(side="left", fill="x", expand=True, padx=(0, 10))
    
    # End indicator (race flag)
    end_label = tk.Label(progress_container, text="🏁", 
                        font=("Helvetica", 14), fg="#ecf0f1", bg="#2c3e50")
    end_label.pack(side="right")
    
    # Progress value label (hidden, just for internal use)
    progress_label = tk.Label(progress_container, text="--", 
                             font=("Helvetica", 1), fg="#2c3e50", bg="#2c3e50")
    progress_label.pack_forget()  # Hide this label
    
    # Start the UI update loop
    update_ui()
    
    # Make the window appear on top initially
    root.lift()
    root.focus_force()
    
    root.mainloop()

# Initialize UI state variables
is_pinned = False
root = None
progress_bar = None
progress_label = None

ui_thread = threading.Thread(target=create_ui, daemon=True)
ui_thread.start()

In [6]:
# --- Load the Trained Model using centralized system ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Try to load the configured model first, with fallback options
model = None
model_name = "unknown"

try:
    # First try the centralized model system
    model = get_model()
    model_name = get_default_model_type()
    
    # Try to load the optimized model weights
    try:
        model.load_state_dict(torch.load('percentage_cnn_optimized.pth', map_location=device))
    except FileNotFoundError:
        model.load_state_dict(torch.load('percentage_cnn.pth', map_location=device))
    
    model = model.to(device)
    
except Exception as e:
    # Fallback to hardcoded model loading
    try:
        # Legacy SimpleCNN fallback
        class SimpleCNN(nn.Module):
            def __init__(self, num_classes=100):
                super(SimpleCNN, self).__init__()
                self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
                self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
                self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
                self.fc1 = nn.Linear(32 * 16 * 16, 512)
                self.fc2 = nn.Linear(512, num_classes)
                self.relu = nn.ReLU()

            def forward(self, x):
                x = self.pool(self.relu(self.conv1(x)))
                x = self.pool(self.relu(self.conv2(x)))
                x = x.view(-1, 32 * 16 * 16)
                x = self.relu(self.fc1(x))
                x = self.fc2(x)
                return x
        
        model = SimpleCNN(num_classes=100).to(device)
        try:
            model.load_state_dict(torch.load('percentage_cnn_optimized.pth', map_location=device))
            model_name = "SimpleCNN (fallback with optimized weights)"
        except FileNotFoundError:
            model.load_state_dict(torch.load('percentage_cnn.pth', map_location=device))
            model_name = "SimpleCNN (fallback with legacy weights)"
    except Exception as fallback_error:
        model = None

if model is not None:
    model.eval()  # Set the model to evaluation mode
    
    # 🚀 PERFORMANCE OPTIMIZATIONS 🚀
    
    # 1. Enable cudnn benchmarking for consistent convolution algorithms
    if device.type == 'cuda':
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
    
    # 2. Disable gradient computation globally (already in eval mode, but this is extra)
    torch.set_grad_enabled(False)
    
    # 3. Try to compile the model with torch.jit for optimization
    try:
        # Create a dummy input for scripting
        dummy_input = torch.randn(1, 1, 64, 64).to(device)
        model = torch.jit.script(model)
        
        # Warm up the compiled model
        for _ in range(5):
            with torch.no_grad():
                _ = model(dummy_input)
        
    except Exception as jit_error:
        pass  # Continue with eager mode
    
    # 4. Set memory allocation strategy
    if device.type == 'cuda':
        torch.cuda.empty_cache()

# --- Define Image Transforms ---
# These must be the same as the transforms used during training
data_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

📋 Using default model type: 'optimized'


In [7]:
from IPython.display import clear_output
textarray = []
dist_box = None

# Create a directory for the dataset
DATASET_DIR = "dataset"
os.makedirs(DATASET_DIR, exist_ok=True)

# CNN confidence threshold - adjust this value based on your model's performance
CONFIDENCE_THRESHOLD = 0.65  # Reset bounding box if confidence is below this

# Pre-allocate tensor for reuse (optimization)
_tensor_cache = None

def predict_with_cnn(image_array):
    """
    Use the trained CNN to predict the percentage from an image array.
    
    Args:
        image_array: numpy array of the preprocessed image
        
    Returns:
        predicted_percentage: integer from 0-99, or None if prediction fails
    """
    global inference_times, avg_inference_time, _tensor_cache
    
    if model is None:
        return None
        
    try:
        # Start timing - more precise timing
        if device.type == 'cuda':
            torch.cuda.synchronize()  # Ensure all previous operations are complete
        inference_start = systime.perf_counter()
        
        # Convert numpy array to PIL Image
        pil_image = Image.fromarray(image_array)
        
        # Apply transforms
        tensor_image = data_transforms(pil_image)
        
        # Reuse tensor cache if possible (optimization)
        if _tensor_cache is None or _tensor_cache.shape[0] != 1:
            _tensor_cache = tensor_image.unsqueeze(0).to(device, non_blocking=True)
        else:
            _tensor_cache.copy_(tensor_image.unsqueeze(0), non_blocking=True)
        
        # Make prediction with minimal overhead
        outputs = model(_tensor_cache)
        _, predicted = torch.max(outputs, 1)
        confidence = torch.softmax(outputs, 1)[0][predicted].item()
        
        # End timing with synchronization
        if device.type == 'cuda':
            torch.cuda.synchronize()  # Wait for GPU operations to complete
        inference_end = systime.perf_counter()
        
        inference_time = (inference_end - inference_start) * 1000  # Convert to ms
        
        # Update inference time tracking
        inference_times.append(inference_time)
        if len(inference_times) > 100:  # Keep only last 100 measurements
            inference_times.pop(0)
        
        # Calculate new average
        new_avg_inference_time = sum(inference_times) / len(inference_times)
        avg_inference_time = new_avg_inference_time
            
        return predicted.item(), confidence
    except Exception as e:
        return None

def the_loop():
    global dist_box, capturing, textarray, camera, percentage, elapsed_ms, total_loops, race_in_progress

    # Start the loop
    while capturing:
        if capturing:
            start_time = systime.perf_counter()
            total_loops += 1
            
            window = camera.get_latest_frame()
            height, width, _ = window.shape
            top_right_region = window[50:height, 0:int(width * 0.35)]

            if dist_box is None:
                # Update race state - searching for race
                race_in_progress = False
                
                preprocessed_region = pre_process(top_right_region)
                results = reader.readtext(preprocessed_region)
                
                dist_found = False
                dist_bbox = None
                dist_index = -1
                
                # Find DIST
                for i, (bbox, text, confidence) in enumerate(results):
                    if "dist" in text.lower() and not dist_found:
                        dist_bbox = np.array(bbox)
                        dist_index = i
                        dist_found = True
                
                # If we found DIST, look for percentage
                if dist_found:
                    dist_x0, dist_y0 = np.min(dist_bbox[:, 0]), np.min(dist_bbox[:, 1])
                    dist_x1, dist_y1 = np.max(dist_bbox[:, 0]), np.max(dist_bbox[:, 1])
                    dist_center_y = (dist_y0 + dist_y1) / 2
                    
                    best_percentage_match = None
                    best_score = 0
                    
                    # Look for percentage indicators with more flexible criteria
                    for j, (bbox, text, confidence) in enumerate(results):
                        if j == dist_index:  # Skip the DIST box itself
                            continue
                            
                        bbox_array = np.array(bbox)
                        nx0, ny0 = np.min(bbox_array[:, 0]), np.min(bbox_array[:, 1])
                        nx1, ny1 = np.max(bbox_array[:, 0]), np.max(bbox_array[:, 1])
                        bbox_center_y = (ny0 + ny1) / 2
                        
                        # More flexible matching criteria
                        text_clean = text.strip().replace(' ', '').replace(',', '').replace('.', '')
                        
                        # Check if it looks like a percentage
                        has_percent = '%' in text_clean
                        has_numbers = any(char.isdigit() for char in text_clean)
                        ends_with_7 = text_clean.endswith('7')  # Sometimes % is read as 7
                        
                        # Position criteria (more flexible)
                        reasonable_y_distance = abs(bbox_center_y - dist_center_y) < 50
                        to_the_right = nx0 > dist_x0
                        reasonable_x_distance = (nx0 - dist_x1) < 200
                        
                        # Calculate a score for this match
                        score = 0
                        if has_percent:
                            score += 50
                        if has_numbers:
                            score += 20
                        if ends_with_7:
                            score += 10
                        if reasonable_y_distance:
                            score += 30
                        if to_the_right:
                            score += 20
                        if reasonable_x_distance:
                            score += 10
                        
                        # Add confidence boost
                        score += confidence * 10
                        
                        if score > best_score and score > 40:
                            best_score = score
                            best_percentage_match = (j, bbox, text, confidence)
                    
                    # If we found a good percentage match
                    if best_percentage_match is not None:
                        j, next_bbox, next_text, next_confidence = best_percentage_match
                        
                        # Calculate combined bounding box
                        next_box = np.array(next_bbox)
                        nx0, ny0 = np.min(next_box[:, 0]), np.min(next_box[:, 1])
                        nx1, ny1 = np.max(next_box[:, 0]), np.max(next_box[:, 1])
                        
                        # Extend bounding box to include both with some padding
                        x0 = int(min(dist_x0, nx0)) - 5
                        y0 = int(min(dist_y0, ny0)) - 5
                        x1 = int(max(dist_x1, nx1)) + 5
                        y1 = int(max(dist_y1, ny1)) + 5
                        
                        # Ensure bounds are within image
                        x0 = max(0, x0)
                        y0 = max(0, y0)
                        x1 = min(top_right_region.shape[1], x1)
                        y1 = min(top_right_region.shape[0], y1)
                    else:
                        # Fallback: just use DIST box with some expansion
                        x0 = int(dist_x0) - 10
                        y0 = int(dist_y0) - 10
                        x1 = int(dist_x1) + 100
                        y1 = int(dist_y1) + 30
                        
                        # Ensure bounds are within image
                        x0 = max(0, x0)
                        y0 = max(0, y0)
                        x1 = min(top_right_region.shape[1], x1)
                        y1 = min(top_right_region.shape[0], y1)
                    
                    # Create the final bounding box
                    dist_box = np.array([[x0, y0], [x1, y0], [x1, y1], [x0, y1]])
            
            clear_output(wait=True)
            
            # If we have the bounding box, crop the image
            if dist_box is not None:
                # Update race state - race is in progress
                race_in_progress = True
                
                roi = top_right_region[int(dist_box[0][1]):int(dist_box[2][1]), int(dist_box[0][0]):int(dist_box[1][0])]
                roi = roi[:, int(roi.shape[1] * 23 / 40):]

                # Preprocess the cropped image for CNN
                preprocessed_region = pre_process_distbox(roi, for_cnn=True)

                # Use CNN for recognition
                cnn_result = predict_with_cnn(preprocessed_region)

            # Process CNN prediction
            try:
                if cnn_result is not None:
                    predicted_percentage, confidence = cnn_result
                    text2 = f"{predicted_percentage}%"
                    
                    percentage = text2

                    # Reset bounding box if confidence is too low
                    if confidence < CONFIDENCE_THRESHOLD:
                        dist_box = None
                else:
                    dist_box = None
            except Exception as e:
                dist_box = None
            
            end_time = systime.perf_counter()
            elapsed_ms = (end_time - start_time) * 1000

            systime.sleep(0.05)

In [ ]:
# Run the main loop
the_loop()